# Data Preparation - Wikipedia Indonesia

Notebook ini mendemonstrasikan proses data preparation pada data tekstual.

## Alur Proses:
1. **Load**: Membaca dan menjelajahi dataset awal
2. **Data Cleaning**: Membersihkan markup Wikipedia dan ekstrak kategori
3. **Pipeline Processing**: Menerapkan transformasi data secara bertahap
4. **Tokenization** : Memecah teks menjadi kunpulan token
5. **Vectorization**: Mengkonversi text menjadi TF-IDF matrix
6. **Search & Retrieval**: Mencari dokumen paling relevan dengan query

In [ ]:
import pandas as pd
import re

## 2. Load Dataset

Membaca file CSV yang berisi data artikel Wikipedia Indonesia dalam format raw (mentah dengan markup Wiki masih ada).

In [ ]:
#Membaca dataframe (file csv), dengan delimiter pipe (|)
df = 

In [ ]:
#menampilkan dataframe

,Unnamed: 0,id,timestamp,title,text
0,0,1,2022-04-24T00:35:51Z,Asam deoksiribonukleat,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,1,3,2022-05-05T11:26:32Z,Anwar Sadat,'''Muhammad Anwar el-Sadat'''; ) adalah seoran...
2,2,5,2022-09-26T04:25:33Z,Arkeologi,[[Berkas:Bulgandry Aboriginal Site.JPG|jmpl|Si...
3,3,6,2022-11-02T02:33:49Z,Antropologi,'''Antropologi''' adalah ilmu tentang manusia....
4,4,8,2022-11-16T04:21:39Z,Bahasa Indonesia,'''Bahasa Indonesia''' adalah [[bahasa nasiona...


In [ ]:
#meilhat informasi umum dari dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86436 entries, 0 to 86435
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  86436 non-null  int64 
 1   id          86436 non-null  int64 
 2   timestamp   86436 non-null  object
 3   title       86436 non-null  object
 4   text        86436 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.3+ MB


#### Melihat contoh isi `teks`

In [1]:
#Menampilkan contoh isi data teks

## 4.Data Cleaning and Extraction

Membersihkan text dari markup Wikipedia dan mengekstrak informasi.
**Fungsi Utama:**
- Ekstrak kategori dari format `[[Kategori:...]]`
- Pembersihan teks lengkap (hapus ref, link, template, dsb)

In [28]:
# mengambil 1 sampel text sebagai contoh
sample_text = df.loc[0, 'text']

### 4.1 Esktrak informasi kategori

`[[Kategori:Genetika molekular|Nukleat asam DNA]]`

`[[Kategori:Asam nukleat]]`

In [29]:
# mencari dan mengekstrak informasi kategori artikel dari teks
x = re.findall(r'\[\[Kategori:(.*?)\]\]', sample_text)
kategori_extracted = []
for kategori in x:
    kategori_extracted.extend(kategori.split('|'))
print(kategori_extracted)

['Genetika molekular', 'Nukleat asam DNA', 'Asam nukleat']


In [30]:
# Modularisasi ke bentuk fungsi, agar lebih mudah digunakan pada dataframe
def extract_kategori(text):
    x = re.findall(r'\[\[Kategori:(.*?)\]\]',text)
    kategori_extracted = []
    for kategori in x:
        kategori_extracted.extend(kategori.split('|'))
    return kategori_extracted

In [ ]:
# menjalankan fungsi terhadap sample_text


['Genetika molekular', 'Nukleat asam DNA', 'Asam nukleat']

### 4.2 Membersihkan/ Menghapus markdown files
`[[Berkas:DNA animation.gif|jmpl|Gambaran tiga dimensi DNA]]{{genetika}}`

In [ ]:
#mencari dan menghapus semua markdown files
sample_text_clean = re.sub(r'\[\[Berkas:.*?\n','', sample_text, flags=re.IGNORECASE)

In [2]:
# menampilkan hasil cleansing

### 4.3 Membersihkan/Menghapus reftag
`<ref>{{cite book|last = Saenger|first = Wolfram|title = Principles of Nucleic Acid Structure|url = https://archive.org/details/...
}}</ref>`

In [ ]:
#mencari dan menghapus semua reftag
sample_text_clean = re.sub(r'<ref.*?<\/ref>', '', sample_text_clean, flags=re.DOTALL)


In [ ]:
# menampilkan hasil cleansing

### 4.4 Membersihkan internal link

`[[Hereditas|pewarisan]]` akan  menjadi `pewarisan` (hanya mengambil labelnya saja)

`[[kromosom]]` akan menjadu `kromosom`

In [ ]:
#cleansing internal link dengan label
sample_text_clean = re.sub(r'\[\[[\w\s]*?\|([\w\s]*?)\]\]',r'\1', sample_text_clean)

In [ ]:
#cleansing internal link tanpa label
sample_text_clean = re.sub(r'\[\[([\w\s]*)\]\]', r'\1', sample_text_clean)

In [ ]:
# menampilkan hasil cleansing

In [36]:
def cleanse_text(text):
    text = re.sub(r'\[\[(Berkas|File).*?\n',' ',text, flags=re.IGNORECASE)
    text = re.sub(r'<ref.*?<\/ref>', ' ', text, flags=re.DOTALL)
    text = re.sub(r'\[\[[\w\s]*?\|([\w\s]*?)\]\]',r'\1',text)
    text = re.sub(r'\[\[(.*?)\]\]', r'\1', text)
    text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
    text = text.strip()
    return text

In [ ]:
# mencoba fungsi pada sample_text dan membandingkan hasil sebelum dan sesudah cleansing


## 5. Eksekusi Data Cleaning Pipeline

Menerapkan semua fungsi pembersihan secara berurutan menggunakan method chaining `.pipe()`.

Output: `df_cleaned` - Dataset yang sudah bersih dengan kolom text_cleaned dan kategori

**Fungsi Pipeline:**
- `start_pipeline()`: Mulai pipeline
- `extract_kategori_pipe()`:  ekstraksi kategori
- `clean_text_pipe()`:  pembersihan text

In [ ]:
# data pipeline
def start_pipeline(df):
    return df.copy()

def extract_kategori_pipe(df):
    df['kategori'] = df['text'].apply(extract_kategori)
    return df

def cleanse_text_pipe(df):
    df['text_cleaned'] = df['text'].apply(cleanse_text)
    return df

In [ ]:
#menjalankan proses pembersihan pada dataframe
df_cleaned = (df
              .pipe(start_pipeline)
              .pipe(extract_kategori_pipe)
              .pipe(cleanse_text_pipe)
              )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86436 entries, 0 to 86435
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    86436 non-null  int64 
 1   id            86436 non-null  int64 
 2   timestamp     86436 non-null  object
 3   title         86436 non-null  object
 4   text          86436 non-null  object
 5   text_cleaned  86436 non-null  object
 6   kategori      86436 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.6+ MB
None


In [ ]:
# menampilkan hasil cleansing dan ekstraksi pada dataframe


,Unnamed: 0,id,timestamp,title,text,text_cleaned,kategori
0,0,1,2022-04-24T00:35:51Z,Asam deoksiribonukleat,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...,Asam deoksiribonukleat lebih dikenal denga...,"[Genetika molekular, Nukleat asam DNA, Asam nu..."
1,1,3,2022-05-05T11:26:32Z,Anwar Sadat,'''Muhammad Anwar el-Sadat'''; ) adalah seoran...,Muhammad Anwar el Sadat adalah seorang p...,"[Presiden Mesir, Perdana Menteri Mesir, Pemena..."
2,2,5,2022-09-26T04:25:33Z,Arkeologi,[[Berkas:Bulgandry Aboriginal Site.JPG|jmpl|Si...,Arkeologi atau ilmu kepurbakalaan ad...,"[Arkeologi, ]"
3,3,6,2022-11-02T02:33:49Z,Antropologi,'''Antropologi''' adalah ilmu tentang manusia....,Antropologi adalah ilmu tentang manusia An...,"[Antropologi, ]"
4,4,8,2022-11-16T04:21:39Z,Bahasa Indonesia,'''Bahasa Indonesia''' adalah [[bahasa nasiona...,Bahasa Indonesia adalah bahasa nasional dan...,"[Bahasa Indonesia, , Bahasa di Indonesia, Bah..."
...,...,...,...,...,...,...,...
86431,86431,3768250,2022-11-19T13:16:22Z,Qatar pada Piala Dunia FIFA,[[Tim nasional sepak bola Qatar]] tidak perna...,Tim nasional sepak bola Qatar tidak pernah lo...,"[Qatar pada Piala Dunia FIFA, , Negara pada P..."
86432,86432,3768386,2022-11-20T04:25:50Z,Bahasa Ma'ya,'''Bahasa Ma'ya''' adalah [[Bahasa Austronesia...,Bahasa Ma ya adalah Bahasa Austronesia dari...,[Rumpun bahasa Halmahera Selatan–New Guinea Ba...
86433,86433,3768388,2022-11-20T04:35:20Z,Nabeul,'''Nabeul (; )''' merupakan kota pesisir yang...,Nabeul merupakan kota pesisir yang te...,"[Permukiman di Tunisia, CS1 sumber berbahasa P..."
86434,86434,3768419,2022-11-20T07:41:32Z,Porsche 963,'''Porsche 963''' adalah mobil balap prototipe...,Porsche 963 adalah mobil balap prototipe ol...,"[Mobil balap Le Mans 24 Jam, Prototipe olahrag..."


## 6. Feature Extraction

- **Tokenisasi** : Merubah teks utuh menjadi komponen-komponen token
- **Vektorisasi**: Mengkonversi text menjadi numerical feature menggunakan TF-IDF (Term Frequency-Inverse Document 

**Output**: TF-IDF matrix - menghasilkan matrix yang merupakan representasi numerik dari setiap teks/dookumen

### 6.1 Tokenisasi

#### Mengapa Tokenisasi Penting?

1. **Pemrosesan Komputer**: Algoritma machine learning memerlukan input yang terstruktur. Tokenisasi mengubah teks yang tidak terstruktur menjadi deretan token yang dapat dianalisis secara sistematis.

2. **Pengurangan Kompleksitas**: Memecah teks besar menjadi satuan kecil memudahkan pemrosesan dan analisis yang lebih efisien.

3. **Ekstraksi Fitur**: Token individual menjadi basis untuk ekstraksi fitur lanjutan seperti TF-IDF, word embeddings, dan analisis frekuensi.

#### Contoh Tokenisasi Sederhana (Word-Level Tokenization)

Teks asli:
```
"Indonesia adalah negara kepulauan terbesar di dunia yang terletak di Asia Tenggara"
```

Setelah tokenisasi dengan memisahkan berdasarkan spasi:
```
['Indonesia', 'adalah', 'negara', 'kepulauan', 'terbesar', 'di', 'dunia', 'yang', 'terletak', 'di', 'Asia', 'Tenggara']
```

Jumlah token: **12 token**

In [ ]:
# simple tokenization
sample_text = 'Indonesia adalah negara kepulauan terbesar di dunia yang terletak di Asia Tenggara'

['Indonesia',
 'adalah',
 'negara',
 'kepulauan',
 'terbesar',
 'di',
 'dunia',
 'yang',
 'terletak',
 'di',
 'Asia',
 'Tenggara']

In [ ]:
#mencoba tokenisasi sederhana (word level)

### 6.2 Vektorisasi Teks (Text Vectorization)

**Vektorisasi** adalah proses fundamental dalam *Natural Language Processing* (NLP) untuk mengubah data teks tidak terstruktur menjadi representasi numerik (vektor) yang dapat dipahami oleh mesin.

Bayangkan vektorisasi sebagai **penerjemah**: Komputer tidak mengerti kata "Apel" atau "Jeruk", tetapi mereka mengerti bahwa `[1, 0]` berbeda dengan `[0, 1]`.

#### Mengapa Vektorisasi Penting?

Komputer bekerja dengan operasi aljabar linier. Tanpa vektorisasi, algoritma machine learning tidak dapat memproses input teks.
1.  **Kompatibilitas Matematis**: Algoritma seperti Regresi, SVM, atau Neural Networks membutuhkan input berupa array angka (matriks).
2.  **Menangkap Makna (Semantik)**: Teknik vektorisasi yang baik tidak hanya mengubah kata jadi angka, tapi juga mempertahankan informasi penting seperti seberapa unik kata tersebut dalam sebuah dokumen.
3.  **Pengukuran Jarak**: Memungkinkan kita menghitung *similarity* (kemiripan) antar dokumen menggunakan rumus geometri seperti *Cosine Similarity* atau *Euclidean Distance*.

---

#### Beberapa Metode Vektorisasi

Berikut adalah beberapa pendekatan populer, diurutkan dari yang paling sederhana hingga metode yang kami pilih:

##### 1. One-Hot Encoding
Setiap kata dalam *vocabulary* diwakili oleh satu bit unik.
* **Konsep**: Jika kita memiliki 1.000 kata unik, setiap kata adalah vektor sepanjang 1.000 dimensi dengan satu angka `1` dan sisanya `0`.
* **Kelemahan**: Menghasilkan dimensi yang sangat besar (*high dimensionality*) dan tidak menangkap frekuensi kata.

##### 2. Bag of Words (BoW) / Count Vectorizer
Metode ini menghitung **frekuensi** kemunculan kata dalam dokumen tanpa memperdulikan urutan.
* **Analogi**: Bayangkan sebuah dokumen dipotong-potong per kata, lalu dimasukkan ke dalam kantong (bag). Kita hanya menghitung jumlah kata "makan" ada berapa, "tidur" ada berapa.
* **Kelemahan**: Kata umum seperti "yang", "dan", "di" akan memiliki nilai (bobot) yang sangat besar, padahal kata-kata tersebut minim informasi unik.

##### 3. TF-IDF (Term Frequency - Inverse Document Frequency)
Metode ini adalah penyempurnaan dari BoW. TF-IDF memberikan bobot pada kata berdasarkan dua prinsip:
1.  **Seberapa sering** kata muncul di dokumen ini? (TF)
2.  **Seberapa jarang** kata muncul di seluruh koleksi dokumen lain? (IDF)

Tujuannya adalah menonjolkan kata yang **sering muncul di satu dokumen tertentu, tapi jarang muncul di dokumen lain** .

---

#### Algoritma TF-IDF

Rumus matematis untuk menghitung bobot kata $t$ dalam dokumen $d$:

$$W_{t,d} = \text{TF}(t,d) \times \text{IDF}(t)$$

Dimana:

1.  **Term Frequency (TF)**: Frekuensi kata dalam dokumen tertentu.
    $$\text{TF}(t,d) = \frac{\text{Jumlah kemunculan kata } t \text{ di dokumen } d}{\text{Total kata dalam dokumen } d}$$

2.  **Inverse Document Frequency (IDF)**: Mengukur seberapa informatif sebuah kata.
    $$\text{IDF}(t) = \log \left( \frac{\text{Total Dokumen } (N)}{\text{Jumlah Dokumen yang mengandung kata } t} \right)$$

**Simulasi Perhitungan Sederhana:**

Misalkan kita memiliki data teks:
* **Doc A**: "Kucing makan ikan"
* **Doc B**: "Kucing tidur"
* **Doc C**: "Ikan berenang"

Nilai bobot untuk kata **"Kucing"** di **Doc A**:

1.  **TF ("Kucing", Doc A)**: Muncul 1 kali dari 3 total kata = $1/3 \approx 0.33$
2.  **IDF ("Kucing")**: Muncul di 2 dokumen (A & B) dari total 3 dokumen.
    $$\text{IDF} = \log(3/2) \approx 0.176$$
3.  **TF-IDF**: $0.33 \times 0.176 = \mathbf{0.058}$

Bandingkan dengan kata **"Berenang"** di **Doc C**:
1.  **TF**: $1/2 = 0.5$
2.  **IDF**: Muncul hanya di 1 dokumen. $\log(3/1) \approx 0.477$
3.  **TF-IDF**: $0.5 \times 0.477 = \mathbf{0.238}$

> **Insight**: Kata "Berenang" memiliki bobot lebih tinggi daripada "Kucing" karena "Berenang" lebih spesifik/langka dalam koleksi dokumen ini.

---

#### Output Vektorisasi: Sparse Matrix

Hasil dari proses ini adalah sebuah matriks di mana:
* **Baris**: Dokumen (Artikel Wikipedia)
* **Kolom**: Kata (Fitur/Vocabulary)
* **Nilai**: Skor TF-IDF

Karena sebagian besar kata tidak muncul di setiap dokumen, matriks ini akan didominasi oleh angka nol. Untuk menghemat memori, ini disimpan sebagai **Sparse Matrix**.

**Ilustrasi Matriks TF-IDF:**

| Dokumen | 'indonesia' | 'adalah' | 'kepulauan' | 'teknologi' | 'ekonomi' |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **Artikel 1** | 0.45 | 0.02 | 0.65 | 0.00 | 0.00 |
| **Artikel 2** | 0.00 | 0.02 | 0.00 | 0.55 | 0.35 |
| **Artikel 3** | 0.30 | 0.01 | 0.00 | 0.00 | 0.75 |

*Catatan: Nilai 0.00 merepresentasikan kata tersebut tidak ada dalam dokumen.*

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
# membuat daftar stop_words


In [ ]:
# vektorisasi menggunakan TfIdf

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 43873033 stored elements and shape (86436, 50000)>

In [ ]:
#mlihat daftar vocabulary


np.int64(21615)

In [ ]:
#melakukan fitting vectorizer pada data

In [ ]:
# melihat representasi nilai sebuah sampel token pada dokumen

np.float32(0.84515274)

### 6.3 Simulasi Search and Retreival Sederhana

Mencoba menemukan dokumen yang paling relevan berdasarkan query (kata kunci)

In [ ]:
# query
query =  ''

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3 stored elements and shape (1, 50000)>

In [ ]:
#melakukan tranformasi pada query (tokenisasi dan vektorisasi)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#menghitung kemiripan query ke kumpulan dokumen
similarity = cosine_similarity(..., ...)

# top 10 dokumen yang paling mirip
top_10_idx = similarity.flatten().argsort()[::-1][:10]

#menampilkan 10 dokumen yang paling mirip
top_10_doc = df_cleaned.loc[top_10_idx, :]

In [ ]:
top_10_doc